In [15]:
import mysql.connector
import pandas as pd
import time
import datetime

In [16]:
cnx = mysql.connector.connect(user='openupadmin', password='6C7E2F760ABFF371',
                              host='openupdb.cxzofl0nts6g.us-east-1.rds.amazonaws.com', port = 3306,
                              database='Openup_db')

In [17]:
cursor = cnx.cursor()

In [10]:
query = ('SELECT id, user_id, url, created_date FROM e_url_map WHERE is_beauty = 1 AND has_thank_you_confirmation = 1 AND user_id > 50577 GROUP BY user_id')

In [11]:
cursor.execute(query)

In [ ]:
data = pd.DataFrame(cursor.fetchall())

In [12]:
data.columns = ['id', 'user_id', 'url', 'created_date']

NameError: name 'data' is not defined

In [ ]:
pasturls = pd.DataFrame(columns = ['id', 'user_id', 'url', 'domain', 'created_date'])

tic = time.time()

for (idx, row) in data.loc[:,['user_id', 'created_date']].iterrows():
    date = row[1]
    user_id = row[0]
    query = ('''SELECT id, user_id, url, domain, created_date FROM e_url_map WHERE is_publisher = 1 AND
    user_id = {0} AND DATE(created_date) BETWEEN DATE('{1}') - INTERVAL 1 DAY AND DATE('{1}')'''.format(user_id, date))
    
    cursor.execute(query)
    tempdata = pd.DataFrame(cursor.fetchall())
    try: 
        tempdata.columns = ['id', 'user_id', 'url', 'domain', 'created_date']
        pasturls = pd.concat((pasturls,tempdata))
    except:
        pass
    
toc = time.time()

In [ ]:
elapsed = toc - tic

In [ ]:
elapsed

In [ ]:
pasturls.groupby('domain').count().to_csv('is_jewelry_domain_counts.csv')

In [ ]:
data[0:5]

In [ ]:
pasturls.to_csv('testpublishers_clinique.csv')

In [ ]:
#pasturls.groupby(['domain','user_id']).count()

In [ ]:
pasturls.groupby(['domain','user_id']).count().to_excel('is_beauty_domain_counts.xlsx')

# AMAZON PURCHASES

In [18]:
query = ('SELECT id, user_id, url, created_date, title FROM e_url_map WHERE title LIKE "Amazon.com Thanks You" AND user_id > 50577')

In [19]:
cursor.execute(query)

In [20]:
amazonbuy = pd.DataFrame(cursor.fetchall())
amazonbuy.columns = ['id', 'user_id', 'url', 'created_date', 'title']

In [21]:
#317 users
len(amazonbuy['user_id'].unique())

#made 2184 purchases
len(amazonbuy)

2221

## URLS THEY SAW BEFORE THE PURCHASE

In [22]:
purchasedata = pd.DataFrame(columns = ['id','user_id','url', 'created_date','title'], index = None)

In [23]:
for (idx,row) in amazonbuy[['user_id', 'created_date']][0:5].iterrows():
    user_id = row[0]
    created_date = row[1]
    query = ('''SELECT id, user_id, url, created_date, title FROM e_url_map WHERE user_id = {0}\
    AND DATE(created_date) BETWEEN DATE('{1}') - INTERVAL 1 HOUR AND DATE('{1}')'''.format(user_id,created_date))
    cursor.execute(query)
    tempdata = pd.DataFrame(cursor.fetchall(), columns = ['id','user_id','url', 'created_date','title'], index=None)
    purchasedata = pd.concat((purchasedata,tempdata))

In [24]:
#purchasedata.to_excel('purchasedata.xlsx')
print(purchasedata)

            id  user_id                                                url  \
0     323443.0  50738.0  https://mail.google.com/mail/u/0/#starred/1575...   
1     323448.0  50738.0  https://www.google.com/url?hl=en&q=http://mail...   
2     323449.0  50738.0  https://mail.google.com/mail/u/0/#starred/1574...   
3     323450.0  50738.0  https://subscribe.hearstmags.com/subscribe/spl...   
4     323451.0  50738.0  https://subscribe.hearstmags.com/subscribe/spl...   
5     323452.0  50738.0  https://subscribe.hearstmags.com/subscribe/off...   
6     323453.0  50738.0          https://mail.google.com/mail/u/0/#starred   
7     323457.0  50738.0             http://open-up.io/extensions/download/   
8     323458.0  50738.0            https://mail.google.com/mail/u/2/#inbox   
9     323459.0  50738.0  https://www.google.com/url?hl=en&q=https://e.h...   
10    323460.0  50738.0  https://mail.google.com/mail/u/2/#inbox/157614...   
11    323461.0  50738.0  https://www.hollisterco.com/webapp/wcs/

## SHOPPING CART GENERATOR FOR USERS

In [ ]:
#cartdict = {x:[pd.DataFrame(columns = ['id','user_id','url', 'created_date','title'])] for x in amazonbuy['user_id']}

In [ ]:
def extractcartitems(user_id):
    #get list of IDs with shopping carts
    query1 = '''SELECT id FROM e_url_map WHERE user_id = {0} and TITLE LIKE
    "Amazon.com Shopping Cart"'''.format(user_id)
    
    cursor.execute(query1)
    ids = [x[0] for x in cursor.fetchall()]
    ids = tuple(ids)
    
    #pull previous rows from above
    
    itemids = []
    for idz in ids:
        query2 = '''SELECT id from e_url_map WHERE
        id = (SELECT MAX(id) from e_url_map WHERE id < {1} AND user_id = {0})'''.format(user_id, idz)
        cursor.execute(query2)
        itemids.append(cursor.fetchall()[0][0])
        
    itemids = tuple(itemids)
    
    
    query3 = '''SELECT id, user_id, url, created_date, title FROM e_url_map WHERE user_id = {0} and id IN {1}'''.format(user_id, itemids)
    cursor.execute(query3)
    
    return pd.DataFrame(cursor.fetchall(), columns = ['id','user_id','url', 'created_date','title'])

In [ ]:
extractcartitems(amazonbuy['user_id'][1])

In [ ]:
amazonbuy['user_id'][0]

# FILTERING SHOPPING URLS

In [ ]:
query = '''SELECT DISTINCT(url), title, domain, created_date FROM e_url_map WHERE has_thank_you_confirmation = 1 AND user_id = 50024'''

In [ ]:
cursor.execute(query)

In [ ]:
userdata = pd.DataFrame(cursor.fetchall(), columns = ['url', 'title', 'domain', 'created_date'])

In [ ]:
userdata.head()

In [ ]:
userdata['created_date'][0] + datetime.timedelta(minutes = +5)

In [ ]:
userdata['created_date'][0] + datetime.timedelta(minutes = 5) < userdata['created_date'][1]

# All user purchase data, return cleaned data

In [ ]:
def main():
    import pandas as pd
    import datetime
    import sys
    
    def clean_checkout_repeats(userdata, delta):
        cleaned_indices = []
        user_index = list(userdata.index)
        for count, index in enumerate(user_index):
            if user_index[count] > -1 and user_index[count] < max(user_index):
                if (userdata['created_date'][user_index[count]] + datetime.timedelta(minutes = delta) < userdata['created_date'][user_index[count + 1]])\
                or (userdata['domain'][user_index[count]] != userdata['domain'][user_index[count+1]]):
                    cleaned_indices.append(user_index[count])

        return userdata.loc[cleaned_indices,:]

    data = pd.read_csv(sys.argv[1], columns = ['user_id', 'url', 'title', 'domain', 'created_date']) 
     
    format = '%Y-%m-%d %H:%M:%S'

    data.loc[:,['created_date']] = [datetime.datetime.strptime(x, format) for x in data['created_date']]
     
    cleaned_data = pd.DataFrame(columns = ['user_id', 'url', 'title', 'domain', 'created_date']) 
     
    for user in data['user_id'].unique():
        pd.concat(cleaned_data,clean_checkout_repeats(data.loc[data['user_id']==user], 5))
        
#if __name__ == "__main__":
#	main()

In [ ]:
query = "SELECT id, user_id, url, title, domain, created_date FROM e_url_map WHERE has_thank_you_confirmation = 1 AND user_id > 50001"

In [ ]:
cursor.execute(query)

In [ ]:
data = pd.DataFrame(cursor.fetchall(),columns = ['id', 'user_id', 'url', 'title', 'domain', 'created_date'])

In [ ]:
# this function takes a series of PURCHASE TIMES and returns the indices of urls that are no less than delta 
# minutes apart

def clean_checkout_repeats(userdata, delta):
    cleaned_indices = []
    user_index = list(userdata.index)
    for count, index in enumerate(user_index):
        if user_index[count] > -1 and user_index[count] < max(user_index):
            if (userdata['created_date'][user_index[count]] + datetime.timedelta(minutes = delta) < userdata['created_date'][user_index[count + 1]])\
            or (userdata['domain'][user_index[count]] != userdata['domain'][user_index[count+1]]):
                cleaned_indices.append(user_index[count])
                
    return userdata.loc[cleaned_indices,:]
    #return indices where row + 1 and -1 > delta

In [ ]:
cleaned_data = pd.DataFrame(columns = ['id', 'user_id', 'url', 'title', 'domain', 'created_date']) 
     
for user in data['user_id'].unique():
    cleaned_data = pd.concat((cleaned_data,clean_checkout_repeats(data.loc[data['user_id']==user], 60)))
    

In [ ]:
len(cleaned_data['user_id'].unique())

# Function to determine average time between domain and purchase

def average_time_to_purchase(domain_of_content, number_of_days_prior, domain_of_purchase) 

CONTENTDOMAIN, PURCHASEDOMAIN, DAYSPRIOR ARE ARGUMENTS

CONTENTDOMAINS={X:0, Y:0, Z:0}

USER_IDS = (FROM PURCHASEDATA W/ PURCHASEDOMAIN)

FOR USER IN USER_IDS:
    USER_TIMES = {X FOR X IN COTENTDOMAINS.KEYS}
    USER_PURCHASE_DATA = SQL QUERY W/ PURCHASEDOMAIN 
    
    FOR PURCHASE IN USER_PURCHASE_DATA:
        LAST_X_DAYS = SQL QUERY W/ DAYSPRIOR
        IF DOMAIN IN LAST_X_DAYS:
            FIND TIMEDIFFERENCE
            APPEND TO USER_TIMES[CONTENTDOMAIN]
            
    FOR DOMAIN IN USER_TIMES, AVERAGE TIME
    
AVERAGE ACROSS ALL USERS

In [ ]:
def average_time_to_purchase(list_of_content_domains, number_of_days_prior, domain_of_purchase):
    contentdomains = {x:None for x in list_of_content_domains}
    
    purchase_query = "SELECT id, user_id, url, title, domain, created_date FROM e_url_map WHERE has_thank_you_confirmation = 1 \
    AND user_id > 50001"
    cursor.execute(purchase_query)
    user_ids = unique(cursor.fetchall())
    
    for user in user_id:
        user_times = {x:None for x in contentdomains.keys()}
        user_purchase_query = "SELECT id, user_id, url, title, domain, created_date FROM e_url_map WHERE \
        has_thank_you_confirmation = 1 AND user_id = {0}".format(user)
        cursor.execute(user_purchase_query)
        user_purchase_data = pd.DataFrame(cursor.fetchall())
        
        for purchase in user_purchase_data['id']:
            last_x_query = ('''SELECT id, user_id, url, created_date, title FROM e_url_map WHERE user_id = {0}\
            AND DATE(created_date) BETWEEN DATE('{1}') - INTERVAL 1 HOUR AND DATE('{1}')'''.format(user_id,created_date))

            cursor.execute(last_x_query)
            last_x_days = pd.DataFrame(cursor.fetchall())
            if domain in ...:
                #FIND DIFFERNECE IN TIME
                user_times['domain'].append(TIME)
                
   #FOR DOMAIN IN USER_TIMES, AVERAGE TIME
    
#AVERAGE ACROSS ALL USERS        

In [ ]:
list_of_content_domains = ['www.elle.com', 'www.nytimes.com', 'www.bbc.com']
number_of_days_prior = 1
purchase_qualifier = 'AND domain LIKE "%amazon.com%'
list_of_purchases = cleaned_data[0:10]

contentdomains = {x:None for x in list_of_content_domains}
        
user_ids = list_of_purchases['user_id'].unique()
    
for user in user_ids:
    user_times = {x:None for x in contentdomains.keys()}
    user_purchases = list_of_purchases.loc[list_of_purchases['user_id']==user]
    for purchase_id,created_date in zip(user_purchases['id'],user_purchases['created_date']):
        last_x_query = ('''SELECT id, user_id, url, created_date, title FROM e_url_map WHERE id = {0}\
        #AND DATE(created_date) BETWEEN DATE('{1}') - INTERVAL 1 HOUR AND DATE('{1}')'''.format(purchase_id,created_date))
        cursor.execute(last_x_query)
        last_x_days = pd.DataFrame(cursor.fetchall())
        

In [ ]:
    for purchase in user_purchase_data['id']:
        last_x_query = ('''SELECT id, user_id, url, created_date, title FROM e_url_map WHERE user_id = {0}\
        AND DATE(created_date) BETWEEN DATE('{1}') - INTERVAL 1 HOUR AND DATE('{1}')'''.format(user_id,created_date))

        #if domain in ...:
            #FIND DIFFERNECE IN TIME
            #user_times['domain'].append(TIME)


In [ ]:
user_purchases.loc[:,['id','created_date']].iterrows()

In [ ]:
last_x_days